In [ ]:
!add-apt-repository -y ppa:alex-p/tesseract-ocr-devel
!apt install -y tesseract-ocr
!apt install tesseract-ocr-fra
!apt install tesseract-ocr-rus
!apt install tesseract-ocr-jpn-vert
!pip install deep-translator

In [ ]:
from google.colab import files
from deep_translator import GoogleTranslator
import cv2
from PIL import Image, ImageDraw, ImageFont
from google.colab.patches import cv2_imshow
from PIL import ImageOps,ImageFilter
import numpy as np
import re
import sys
import io


for uploaded_file in files.upload().values():
    img = cv2.imdecode(np.frombuffer(uploaded_file, np.uint8), cv2.IMREAD_COLOR)
    image = Image.fromarray(img)
    gray_image = ImageOps.grayscale(image)
    font = ImageFont.truetype("/content/animeacev02italicbold.ttf", 10)
    threshold_value = 100
    binarized_image = gray_image.point(lambda pixel: 0 if pixel < threshold_value else 255, mode="L")
    filtered_array = np.array(binarized_image)
    cv2.imwrite('temp.jpg', filtered_array)
    img1 = cv2.imread('temp.jpg')
    pattern = r"[!@#$%^&*()\-_=+{}\[\]|\\:;\"'<>?,./—]"
    !tesseract "temp.jpg" output -l eng  --psm 11  tsv 
    pil_img = Image.fromarray(img)
    draw = ImageDraw.Draw(pil_img)
    with open('output.tsv', 'r') as f:
        lines = f.readlines()[1:]
        for line in lines:
            try:
                level, page_num, block_num, par_num, line_num, word_num, left, top, width, height, conf, text = line.strip().split('\t')
                left, top, width, height = int(left), int(top), int(width), int(height)
                bottom_right = (left + width, top + height)
                
                if float(conf) > 55  and not re.search(pattern, text)  :
                   
                    if not any(char.isdigit() for char in text):
                        translated = GoogleTranslator(source='auto', target='ru').translate(text)
                    draw.rectangle((left, top, left + width, top + height), outline=(0, 255, 0), width=2)
                    polygon_points = [(left, top), (left + width, top), (left + width, top + height), (left, top + height)]
                    draw.polygon(polygon_points, fill=(255, 255, 255))
                    if translated is not None :
                      
                        draw.text((left, top), translated,font=font, fill=(255, 0, 0))
            except ValueError:
                continue
    img1 = np.array(pil_img)
    cv2_imshow( img1)
    cv2.waitKey(0)
    cv2.destroyAllWindows()